In [1]:
# Imports

import pandas as pd
import numpy as np
import time
import plotly.express as px
import random


In [2]:
# Eight Queens State

class EightQueensState:
    def __init__(self, board_size, state=None):
        if state is None:
            # Inital board set up
            self.state = np.random.randint(0, board_size, board_size)
        else:
            self.state = np.array(state) 

        # Set the goal fitness value where there is no queen attacks 
        self.goal_fitness = int(((board_size ** 2) - board_size) / 2)
    
    
    def fitness(self):
        count = 0
        n = len(self.state)
        
        for i in range(n - 1):
            remaining_queens = self.state[i + 1:]
            current_queen = self.state[i]
            
            # Count of the queen attacks on the row 
            count += (current_queen == remaining_queens).sum()
            
            # Count of the queen attacks on the diagonal 
            distances = np.arange(1, n - i)
            upper_diagonal = current_queen + distances
            lower_diagonal = current_queen - distances
            
            # Total queen on queen attacks
            count += sum((remaining_queens == upper_diagonal) | (remaining_queens == lower_diagonal))

        # Return the final fitness 
        return self.goal_fitness - count

In [3]:
# Plot GA

def plot_ga(fitness_history):

    # Create DataFrame
    df = pd.DataFrame({
        'Generation': range(len(fitness_history)),
        'Fitness': fitness_history
    })

    # Create the figure with dark theme using plotly express
    fig = px.line(
        df,
        x='Generation',
        y='Fitness',
        title='Eight Queens - Genetic Algorithm Fitness Evolution',
        labels={'Fitness': 'Fitness Score (28 = Perfect Solution)', 'Generation': 'Generation'},
        template='plotly_dark'
    )

    # Add the solution point as a marker if found
    solution_point = df[df['Fitness'] == 28]
    if not solution_point.empty:
        fig.add_scatter(
            x=solution_point['Generation'],
            y=solution_point['Fitness'],
            mode='markers',
            name='Solution Found',
            marker=dict(color='#00FF00', size=15, symbol='star')
        )

    # Add reference line for perfect fitness
    fig.add_hline(
        y=28, 
        line_dash="dash", 
        line_color="rgba(0, 255, 0, 0.3)",
        annotation_text="Perfect Fitness (28)",
        annotation_position="right",
        annotation_font_color="white"
    )

    # Add annotations for initial and final fitness
    fig.add_annotation(
        x=0,
        y=fitness_history[0],
        text=f"Initial Fitness: {fitness_history[0]}",
        showarrow=True,
        arrowhead=1,
        ax=-50,
        ay=-30,
        font=dict(size=14, color='#00B5F7'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    if solution_point.empty:
        final_gen = len(fitness_history) - 1
        final_fitness = fitness_history[-1]
        annotation_text = f"Final Fitness: {final_fitness}"
    else:
        final_gen = solution_point['Generation'].iloc[0]
        annotation_text = "Solution Found!"

    fig.add_annotation(
        x=final_gen,
        y=fitness_history[final_gen],
        text=annotation_text,
        showarrow=True,
        arrowhead=1,
        ax=50,
        ay=-50,
        font=dict(size=14, color='#00FF00'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    # Final adjustments
    fig.update_layout(
        xaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False),
        yaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False, range=[min(fitness_history) - 1, 29]),
        height = 700,
        width=1500,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor='rgba(50, 50, 50, 0.8)'
        ),
        hovermode='x unified',
        margin=dict(t=100, b=50, l=50, r=50)
    )

    print(f"Solution found in {final_gen + 1} generations")
    print(f"Final fitness: {fitness_history[final_gen]}")
    fig.show()

In [36]:
# Genetic Algorithm

class ExtendedGeneticAlgorithm:
    def __init__(self, board_size, population_size, max_generations, tournament_size, initial_mutation_rate, initial_population, stale_count):
        self.board_size = board_size
        self.population_size = population_size
        self.mutation_probability = initial_mutation_rate
        self.last_state = None
        self.previous_best_fitness = 0
        self.goal_fitness = int(((board_size ** 2) - board_size) / 2)
        self.max_generations = max_generations
        self.tournament_size = tournament_size
        self.stale_count = stale_count
        self.possible_positions = {i: list(set(range(board_size)) - {i}) for i in range(board_size)}

        if initial_population == 'Random':
            # Initial population - all random individuals
            self.current_population = [EightQueensState(board_size) for _ in range(population_size)]

        elif initial_population == 'Neighbors':
            # Initial population - unique neighbors of a random initial state
            initial_state = np.random.randint(0, board_size, board_size)
            self.current_population = [EightQueensState(board_size, self.random_neighbour(initial_state)) for _ in range(population_size)]


    def random_neighbour(self, state):
        new_state = state.copy() 

        # Choose a random column
        column = np.random.choice(range(self.board_size)) 

        # Create a list of possible row positions excluding the current row of the queen
        possible_rows = [row for row in range(self.board_size) if row != state[column]]

        # Set the queen in the chosen column to a new row position
        new_state[column] = np.random.choice(possible_rows)

        return new_state

    # Extension
    def tournament_selection(self, population_data):
        # Tournament selection implementation
        tournament = random.sample(population_data, self.tournament_size)
        return max(tournament, key=lambda x: x['fitness'])['state']


    def routlette_wheel_selection(self, population_data):
        # Generate a random number between 0 and 1
        random_value = np.random.random()
        
        # Use cumulative probability to select an individual
        cumulative_probability = 0.0
        for individual in population_data:
            cumulative_probability += individual['selection_prob']
            if cumulative_probability > random_value:
                return individual['state']



    # # Extension
    # def improved_crossover(self, parent1, parent2):
    #     # Multi-point crossover
    #     points = sorted(random.sample(range(1, len(parent1)), 2))
    #     child1 = np.concatenate([
    #         parent1[:points[0]],
    #         parent2[points[0]:points[1]],
    #         parent1[points[1]:]
    #     ])
    #     child2 = np.concatenate([
    #         parent2[:points[0]],
    #         parent1[points[0]:points[1]],
    #         parent2[points[1]:]
    #     ])
    #     return child1, child2


    def crossover(self, parent1, parent2):
        # Perform a single-point crossover to produce two offspring via randomisation
        idx = np.random.randint(1, len(parent1))  

        # Optimisaiton 
        idx = 4

        # Apply the cross over of two parents to product two children 
        return np.concatenate([parent1[:idx], parent2[idx:]]), np.concatenate([parent2[:idx], parent1[idx:]])


    # # Extension
    # def calculate_diversity(self):
    #     # Calculate population diversity using Hamming distance
    #     diversity = 0
    #     for i in range(len(self.current_population)):
    #         for j in range(i+1, len(self.current_population)):
    #             diversity += np.sum(self.current_population[i].state != 
    #                               self.current_population[j].state)
    #     return diversity / (len(self.current_population) * (len(self.current_population) - 1))
    

    def mutate(self, state):
        # Randomly mutate a queen's position in one column with a certain probability
        if np.random.random() < self.mutation_probability:

             # Select a random column to mutate
            idx = np.random.randint(len(state))  

            # Randomly choose a new row position for the queen in the selected column, excluding current row 
            state[idx] = np.random.choice(list(set(range(self.board_size)) - {state[idx]}))
        return state


    def create_next_generation(self, population_data):
        next_generation = []
        
        # Optimization 1: Keep top performer without mutation
        # next_generation.append(EightQueensState(self.board_size, population_data[0]['state']))
        
        # Optimization 2: Pre-select parents in batches for speed
        num_pairs_needed = (self.population_size - 1) // 2
        parent_pairs = []
        
        # Get cumulative probabilities once
        cum_probs = []
        cum_sum = 0
        for individual in population_data:
            cum_sum += individual['selection_prob']
            cum_probs.append(cum_sum)
        
        # Batch selection of random values for speed
        random_values = np.random.random(num_pairs_needed * 2)
        
        # Fast parent selection
        for i in range(0, len(random_values), 2):
            parent1_idx = np.searchsorted(cum_probs, random_values[i])
            parent2_idx = np.searchsorted(cum_probs, random_values[i+1])
            parent_pairs.append((
                population_data[parent1_idx]['state'],
                population_data[parent2_idx]['state']
            ))
        
        # Optimization 3: Vectorized crossover and mutation
        for parent1, parent2 in parent_pairs:
            # Fixed crossover point at 4 as you found optimal
            child1 = np.concatenate([parent1[:4], parent2[4:]])
            child2 = np.concatenate([parent2[:4], parent1[4:]])
            
            # Faster mutation using pre-calculated positions
            if np.random.random() < self.mutation_probability:
                idx = np.random.randint(len(child1))
                child1[idx] = np.random.choice(self.possible_positions[child1[idx]])
            if np.random.random() < self.mutation_probability:
                idx = np.random.randint(len(child2))
                child2[idx] = np.random.choice(self.possible_positions[child2[idx]])
            
            next_generation.extend([
                EightQueensState(self.board_size, child1),
                EightQueensState(self.board_size, child2)
            ])
        
        self.current_population = next_generation[:self.population_size]


    def calculate_population_fitness(self):
        # Optimization 4: Faster fitness calculation
        population_data = []
        total_fitness = 0
        
        # Pre-allocate arrays for speed
        fitness_values = np.zeros(len(self.current_population))
        
        for i, individual in enumerate(self.current_population):
            fitness = individual.fitness()
            fitness_values[i] = fitness
            total_fitness += fitness
        
        # Vectorized probability calculation
        selection_probs = fitness_values / total_fitness
        
        # Create population data in one go
        population_data = [{
            'state': individual.state,
            'fitness': fitness_values[i],
            'selection_prob': selection_probs[i]
        } for i, individual in enumerate(self.current_population)]
        
        return sorted(population_data, key=lambda x: x['fitness'], reverse=True)


    def stale_fitness(self, fitness):
    # Check if there are at least 10 items in the list
        if len(fitness) < self.stale_count:
            return False
        # Slice the last x items and convert to a set
        return len(set(fitness[-self.stale_count:])) == 1

    def run(self):
        generation = 0
        fitness_history = []
        fitness_staleness = 0
        
        while generation < self.max_generations:
            population_data = self.calculate_population_fitness()
            # current_diversity = self.calculate_diversity()
            
            # Store history
            best_fitness = population_data[0]['fitness']
            fitness_history.append(best_fitness)
            # diversity_history.append(current_diversity)

            # if generation > 100:
            #     idx = len(fitness_history) - 1
            #     if best_fitness == fitness_history[idx - 10]:
            #         self.current_population = self.inital_population
            #         # generation = 0


            # if self.stale_fitness(fitness_history):
            #     if self.mutation_probability != 0.3:
            #         print('Stale count hit')
            #         self.mutation_probability = 0.3
            # else:
            #     self.mutation_probability = 0.15


            # # Adapt mutation rate
            # if generation != 0 and (best_fitness + 2) < self.previous_best_fitness:
            #     self.mutation_probability = 0.3
            #     print('adpative mutation applied', best_fitness, self.previous_best_fitness)
            
            # self.previous_best_fitness = best_fitness
    
    
            if best_fitness == self.goal_fitness:
                self.last_state = population_data[0]['state']
                # print(f"Solution found in generation {generation}")
                # print(f"Final mutation rate: {self.mutation_probability:.3f}")
                # print(f"Final diversity: {current_diversity:.3f}")
                return fitness_history, generation
            
            self.create_next_generation(population_data)
            generation += 1
        
        return fitness_history, generation
    

    def display_solution(self):
        # Display the chessboard with queens in the positions defined by self.last_state
        for row in range(self.board_size):
            print(f'{self.board_size - row} |', end=' ')
            for col in range(self.board_size):
                if self.last_state[col] == row:
                    print('Q ', end='')  
                else:
                    print('- ', end='') 
            print(f'|')

In [53]:
class ExtendedGeneticAlgorithm2:
    def __init__(self, board_size, population_size, max_generations, initial_mutation_rate, initial_population):
        self.board_size = board_size
        self.population_size = population_size  
        self.mutation_probability = initial_mutation_rate
        self.max_generations = max_generations
        self.goal_fitness = int(((board_size ** 2) - board_size) / 2)
        
        # Dynamic mutation rate based on board size
        self.base_mutation_rate = 1/board_size
        self.high_mutation_rate = 0.3
        self.low_mutation_rate = 0.1
        
        # Smart initialization
        if initial_population == 'Smart':
            # Create semi-intelligent starting positions
            self.current_population = []
            for _ in range(population_size):
                # Place queens with minimum one space between them
                state = np.zeros(board_size, dtype=int)
                for col in range(board_size):
                    available_rows = set(range(board_size))
                    if col > 0:
                        # Avoid same row and adjacent rows from previous queen
                        available_rows -= {state[col-1], state[col-1]+1, state[col-1]-1}
                    state[col] = np.random.choice(list(available_rows))
                self.current_population.append(EightQueensState(board_size, state))
        else:
            self.current_population = [EightQueensState(board_size) for _ in range(population_size)]



    def adaptive_parameters(self, generation, best_fitness, fitness_history):
        # Adaptive mutation rate based on progress
        if len(fitness_history) >= 10:
            if len(set(fitness_history[-10:])) == 1:  # Stagnation
                self.mutation_probability = self.high_mutation_rate
            else:
                self.mutation_probability = self.low_mutation_rate
        
        # Increase mutation rate if stuck at local optimum
        if best_fitness < self.goal_fitness - 2 and generation > 50:
            self.mutation_probability = self.high_mutation_rate
            
        return self.mutation_probability



    def create_next_generation(self, population_data):
        next_generation = []
        
        # Enhanced elitism - keep top 10% performers
        elite_size = self.population_size // 10
        elite = [EightQueensState(self.board_size, x['state']) for x in population_data[:elite_size]]
        next_generation.extend(elite)
        
        # Create rest of population with bias towards better solutions
        while len(next_generation) < self.population_size:
            # Bias parent selection towards top performers
            if np.random.random() < 0.7:  # 70% chance to pick from top half
                parent1 = self.routlette_wheel_selection(population_data[:len(population_data)//2])
                parent2 = self.routlette_wheel_selection(population_data[:len(population_data)//2])
            else:
                parent1 = self.routlette_wheel_selection(population_data)
                parent2 = self.routlette_wheel_selection(population_data)
            
            child1, child2 = self.crossover(parent1, parent2)
            
            # Smart mutation - avoid obvious conflicts
            child1 = self.smart_mutate(child1)
            child2 = self.smart_mutate(child2)
            
            next_generation.extend([
                EightQueensState(self.board_size, child1),
                EightQueensState(self.board_size, child2)
            ])
        
        self.current_population = next_generation[:self.population_size]



    def smart_mutate(self, state):
        if np.random.random() < self.mutation_probability:
            # Find worst conflicting queen
            conflicts = np.zeros(len(state))
            for i in range(len(state)):
                for j in range(len(state)):
                    if i != j:
                        if state[i] == state[j] or \
                           abs(state[i] - state[j]) == abs(i - j):
                            conflicts[i] += 1
            
            # Mutate queen with most conflicts
            worst_queen = np.argmax(conflicts)
            
            # Find best position for this queen
            best_pos = state[worst_queen]
            min_conflicts = conflicts[worst_queen]
            
            for pos in range(self.board_size):
                if pos != state[worst_queen]:
                    conflicts = 0
                    for j in range(len(state)):
                        if j != worst_queen:
                            if pos == state[j] or abs(pos - state[j]) == abs(worst_queen - j):
                                conflicts += 1
                    if conflicts < min_conflicts:
                        best_pos = pos
                        min_conflicts = conflicts
            
            state[worst_queen] = best_pos
        return state
    


    def routlette_wheel_selection(self, population_data):
        # Generate a random number between 0 and 1
        random_value = np.random.random()
        
        # Calculate total fitness for this subset
        total_fitness = sum(x['fitness'] for x in population_data)
        
        # Calculate selection probabilities
        cumulative_probability = 0.0
        for individual in population_data:
            probability = individual['fitness'] / total_fitness
            cumulative_probability += probability
            if cumulative_probability > random_value:
                return individual['state']
        
        # Fallback
        return population_data[-1]['state']



    def crossover(self, parent1, parent2):
        # Fixed crossover point at 4 as found optimal
        return np.concatenate([parent1[:4], parent2[4:]]), np.concatenate([parent2[:4], parent1[4:]])



    def calculate_population_fitness(self):
        # Calculate fitness and selection probability for each individual
        population_data = []
        total_fitness = 0
        
        for individual in self.current_population:
            fitness = individual.fitness()
            population_data.append({
                'state': individual.state,
                'fitness': fitness
            })
            total_fitness += fitness
        
        # Calculate selection probability for each individual
        for data in population_data:
            data['selection_prob'] = data['fitness'] / total_fitness
            
        return sorted(population_data, key=lambda x: x['fitness'], reverse=True)



    def run(self):
        generation = 0
        fitness_history = []
        
        while generation < self.max_generations:
            population_data = self.calculate_population_fitness()
            
            # Store best fitness
            best_fitness = population_data[0]['fitness']
            fitness_history.append(best_fitness)
            
            # Adapt parameters
            self.adaptive_parameters(generation, best_fitness, fitness_history)
            
            # Check if solution found
            if best_fitness == self.goal_fitness:
                self.last_state = population_data[0]['state']
                return fitness_history, generation
            
            self.create_next_generation(population_data)
            generation += 1
        
        return fitness_history, generation

In [55]:
# Run

def run_multiple_trials(trials):
    all_times = []
    all_generations = []
    successes = 0
    max_generations = 1000  # Define a max generation limit to consider a run successful

    fitness_histories = []
    # diversity_histories = []

    for trial in range(trials):
        # print(f"\nTrial {trial + 1}:")
        
        start_time = time.time()
        
        # ga = ExtendedGeneticAlgorithm(
        #     board_size=8,
        #     population_size=250,
        #     initial_mutation_rate=0.15,
        #     tournament_size = 0,
        #     max_generations=1000,
        #     initial_population='Neighbors',
        #     stale_count=50
        # )

        ga = ExtendedGeneticAlgorithm2(
            board_size=8,
            population_size=200,  # Larger population
            max_generations=1000,
            initial_mutation_rate=0.15,
            initial_population='Dumb'
        )
                
        fitness_history, generations = ga.run()  # Assume `run()` returns (fitness_history, generations)

        fitness_histories.append(fitness_history)
        # diversity_histories.append(diversity_history)
        
        end_time = time.time()
        time_taken = end_time - start_time
        all_times.append(time_taken)
        all_generations.append(generations)
        
        if generations < max_generations:
            successes += 1  # Count as success if solution found within max_generations
        
        print(f"Solution found in: {time_taken:.2f} seconds, Generations: {generations}")

    # Calculate and print metrics
    avg_time = np.mean(all_times)
    std_time = np.std(all_times)
    avg_generations = np.mean(all_generations)
    std_generations = np.std(all_generations)
    cv_generations = std_generations / avg_generations if avg_generations > 0 else float('inf')
    success_rate = successes / trials * 100
    max_time = np.max(all_times)
    min_time = np.min(all_times)

    # print("\n--- Performance Metrics ---")
    print(f"Average Time: {avg_time:.2f} seconds")
    print(f"Standard Deviation of Time: {std_time:.2f} seconds")
    print(f"Max Time: {max_time:.2f} seconds")
    print(f"Min Time: {min_time:.2f} seconds")
    # print(f"Average Generations to Solution: {avg_generations:.2f}")
    # print(f"Standard Deviation of Generations: {std_generations:.2f}")
    # print(f"Coefficient of Variation of Generations: {cv_generations:.2f}")
    # print(f"Success Rate: {success_rate:.2f}%")

    return fitness_histories

# Run the multiple trials
fitness_histories = run_multiple_trials(10)

Solution found in: 0.12 seconds, Generations: 9
Solution found in: 0.09 seconds, Generations: 8
Solution found in: 0.10 seconds, Generations: 9
Solution found in: 0.06 seconds, Generations: 4
Solution found in: 0.03 seconds, Generations: 2
Solution found in: 0.07 seconds, Generations: 6
Solution found in: 0.16 seconds, Generations: 14
Solution found in: 0.06 seconds, Generations: 5
Solution found in: 0.08 seconds, Generations: 7
Solution found in: 0.06 seconds, Generations: 5
Average Time: 0.08 seconds
Standard Deviation of Time: 0.04 seconds
Max Time: 0.16 seconds
Min Time: 0.03 seconds


In [49]:
for fitness_history in fitness_histories:
    plot_ga(fitness_history)

Solution found in 3 generations
Final fitness: 28


Solution found in 9 generations
Final fitness: 28


Solution found in 11 generations
Final fitness: 28


Solution found in 4 generations
Final fitness: 28


Solution found in 5 generations
Final fitness: 28


Solution found in 6 generations
Final fitness: 28


Solution found in 3 generations
Final fitness: 28


Solution found in 2 generations
Final fitness: 28


Solution found in 4 generations
Final fitness: 28


Solution found in 4 generations
Final fitness: 28
